In [1]:
words = open('names.txt', 'r').read().splitlines()

In [2]:
import torch
import torch.nn.functional as F

In [3]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}

In [4]:
N = torch.zeros((27, 27, 27), dtype=torch.int32)
for w in words:
    chs = ['.'] + ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        id1, id2, id3 = stoi[ch1], stoi[ch2], stoi[ch3]
        N[id1, id2, id3] += 1

In [5]:
P = (N+1).float()
P = P / P.sum(2, keepdim=True)
g = torch.Generator().manual_seed(2147483647 + 2)

# Name generator
for x in range(20):

    i1 = 0
    i2 = 0
    out = []
    while True:
        next = torch.multinomial(P[i1, i2], num_samples=1, replacement=True, generator=g).item()

        if next == 0:
            break
        i1 = i2
        i2 = next
        out.append(itos[next])
    print(''.join(out))


taylor
keigelielillagaurjraidalana
marqoken
lonniah
zamdennet
artymaya
que
kenna
mond
jaarnishilleschamilynneifen
douriyiclix
herron
kelly
joan
aun
ron
ka
orallee
marameldca
jasricatacotndricdtkhyia


In [6]:
x1 = []
x2 = []
ys = []
num = 0
N = torch.zeros((27, 27, 27), dtype=torch.int32)

for w in words:
    chs = ['.', '.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        id1 = stoi[ch1]
        id2 = stoi[ch2]
        id3 = stoi[ch3]

        x1.append(id1)
        x2.append(id2 + 27)
        ys.append(id3)

        N[id1, id2, id3] += 1

x1 = torch.tensor(x1)
x2 = torch.tensor(x2)
ys = torch.tensor(ys)
num = ys.nelement()

P = (N+1).float()
P = P / P.sum(2, keepdim=True)

g = torch.Generator().manual_seed(2147483647)
W = torch.randn((54, 27), generator=g, requires_grad=True)

In [13]:
for k in range(100):
    logits = W[x1] + W[x2]
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    loss = F.cross_entropy(logits, P[x1, x2 - 27])

    W.grad = None
    loss.backward()

    W.data += -50 * W.grad

print(loss.item())

2.526679277420044


In [14]:
# Testing Code

for k in range(10):
    input = torch.zeros(54)
    f = 0
    s = 27

    out = []
    while True:
        logits = W[f] + W[s]
        counts = logits.exp()
        probs = counts / counts.sum()

        idx = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()

        if idx == 0:
            break
        out.append(itos[idx])

        f = s - 27
        s = idx + 27
        
    print(''.join(out))

kadely
naryusariegs
jahlbere
dani
amiyanren
kadien
wayahientrikhev
tue
pee
malka
